In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import train_test_split
from sklearn.feature_selection import VarianceThreshold
from scipy import stats
from sknn.mlp import Classifier, Layer
from sklearn.grid_search import RandomizedSearchCV

import time

def get_accuracy(prediction,actual):
    assert(len(prediction)==len(actual))
    
    correct = 1.0*np.count_nonzero(prediction==actual)
    total = 1.0*(len(prediction))
    
    print "The accuracy is: "+str(correct/total)
    return correct/total

In [16]:
#X_train=pd.read_excel(r'H:\Harvard ML\practice 2\SCfeatures_train.xlsx')
#t_train=X_train['t_train']
#X_train=X_train.drop('t_train',1)
#X_test=pd.read_csv(r'H:\Harvard ML\practice 2\SCfeatures_test.csv')
#X_test=X_test[X_train.columns]

X_train=pd.read_csv(r'C:\Users\Yohann\Documents\Machine Learning\practical2\X_train.csv')
t_train=pd.read_csv(r'C:\Users\Yohann\Documents\Machine Learning\practical2\t_train.csv')
train_ids=pd.read_csv(r'C:\Users\Yohann\Documents\Machine Learning\practical2\train_ids.csv')
X_test=pd.read_csv(r'C:\Users\Yohann\Documents\Machine Learning\practical2\X_test.csv')
t_test=pd.read_csv(r'C:\Users\Yohann\Documents\Machine Learning\practical2\t_test.csv')
test_ids=pd.read_csv(r'C:\Users\Yohann\Documents\Machine Learning\practical2\test_ids.csv')

#t_train must be a list of integers not labels

In [22]:
print X_train.shape
print X_test.shape
print t_train.shape
print t_test.shape
print train_ids.shape
print test_ids.shape


(3086, 119)
(3724, 119)
(3086, 1)
(3724, 1)
(3724, 1)
(3086, 1)


In [59]:
#t_train70.as_matrix().reshape(len(t_train70),)
t_train70.as_matrix().reshape(len(t_train70),).shape
#X_train70.as_matrix().shape

(2160L,)

In [23]:
#prepare validation set
X_train70, X_train30, t_train70, t_train30 = train_test_split(X_train, t_train, test_size=0.3)
n_targets=t_train.max()+1

In [61]:
from sklearn import linear_model

logreg = linear_model.LogisticRegression()
logreg.fit(X_train70, t_train70.as_matrix().reshape(len(t_train70),))
prediction_log = logreg.predict(X_train30)

get_accuracy(prediction_log,t_train30.as_matrix().reshape(len(t_train30),))

The accuracy is: 0.718142548596


0.7181425485961123

In [ ]:
#### TEST WITH AND WITHOUT DUMMIES

In [62]:
#from sknn.platform import cpu32, thread16
from sknn.mlp import Classifier, Layer
start_time = time.time()

nn = Classifier(
    layers=[
        Layer("Sigmoid",warning=None, units=50),
        #Layer("Sigmoid",warning=None, units=200),
        #Layer("Sigmoid",warning=None, units=200),
        #Layer("Sigmoid",warning=None, units=200),
        Layer("Softmax",warning=None, units=n_targets)],
    learning_rate=0.001,
    n_iter=20)
nn.fit(X_train70.as_matrix(), t_train70.as_matrix().reshape(len(t_train70),))

#prediction_nn=nn.predict(X_train30)
             
#prediction_nn=prediction_nn.reshape(len(X_train30),) 

#print("--- %s seconds ---" % round((time.time() - start_time),1))                         

#get_accuracy(prediction_nn,t_train30.values)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
def write_predictions(test_ids,prediction):
    assert(len(prediction)==len(test_ids))
    
    writer = csv.writer(open(r'H:\Harvard ML\practice 2\predictionNN.csv','wb'))
    
    writer.writerow(["Id","Prediction"])
    for i in range(0,len(test_ids)):
        writer.writerow([test_ids[i],prediction[i]])

write_predictions(test_ids,prediction)

In [71]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print("Classification report for classifier %s:\n%s\n" % (nn, classification_report(t_train30.values, prediction_nn)))
print("Confusion matrix:\n%s" % confusion_matrix(t_train30.values, prediction_nn))

Classification report for classifier Classifier(batch_size=1, callback=None, debug=False, dropout_rate=None,
      f_stable=0.001,
      hidden0=<sknn.nn.Layer `Sigmoid`: units=100, name=u'hidden0', frozen=False>,
      layers=[<sknn.nn.Layer `Sigmoid`: units=100, name=u'hidden0', frozen=False>, <sknn.nn.Layer `Softmax`: units=15, name=u'output', frozen=False>],
      learning_momentum=0.9, learning_rate=0.001, learning_rule=u'sgd',
      loss_type=None, n_iter=50, n_stable=10,
      output=<sknn.nn.Layer `Softmax`: units=15, name=u'output', frozen=False>,
      random_state=None, regularize=None, valid_set=None, valid_size=0.0,
      verbose=None, warning=None, weight_decay=None, weights=None):
             precision    recall  f1-score   support

          0       0.12      0.03      0.05        30
          1       0.67      0.33      0.44        12
          2       1.00      0.31      0.47        13
          3       0.00      0.00      0.00        10
          4       1.00      0

In [ ]:
'''
rs = RandomizedSearchCV(nn, param_distributions={
    'learning_rate': stats.uniform(0.001, 0.05),
    'hidden0__units': stats.randint(4, 12),
    'hidden0__type': ["Rectifier", "Sigmoid", "Tanh"]})
rs.fit(X_train, y_train)

rs.best_estimator_

prediction_rs=rs.predict(X_test)
'''

In [ ]:
def EXRT(X_train,t_train,x,t,predict):

	clf = ExtraTreesClassifier(n_estimators=500, max_depth=None)
	clf.fit(X_train70, t_train70)
	prediction = clf.predict(X_train30)

	if predict:
		write_predictions(t,prediction)
	else:
		get_accuracy(prediction,t)

For prediction (will result in a prediction file)
 
models.EXRT(X_train,t_train,X_test,test_ids,True)
 
For cross-validation (will print accuracy)
 
models.EXRT(X_train,t_train,X_valid,t_valid,False)
